# Twitter Data Analysis

Prerequisite:

Creating a Twitter application using the Twitter API

Approach:

For our approach I have created Spark parquet files by extracting tweets in real time from Twitter. 
The below code 
1. extracts tweets , location and user handle. (for our apprach the #jobsearch is used as the criterion, other filters can be changed based on requirement) 
2. performs sentimental analysis using the Vader Algorithm.
3. Create Parquet files which is highly effecient and quick in data extraction using SparkSQL
4. Export data as CSV for further analysis

In [19]:
import os
import re
import sys
import nltk
import json
import math
import time
import googlemaps
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from pyspark.sql import SparkSession
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from twitter import Twitter, OAuth, TwitterHTTPError, TwitterStream


In [15]:
# create Sentiment analyzer
sa = SentimentIntensityAnalyzer()

In [16]:
def retrieve(): 
        #Return data from Twitter using twitter application credentials
        ACCESS_TOKEN = ''
        ACCESS_SECRET = ''
        CONSUMER_KEY = ''
        CONSUMER_SECRET = ''
        #Read train data with sentiment positive and negative
        oauth = OAuth(ACCESS_TOKEN, ACCESS_SECRET, CONSUMER_KEY, CONSUMER_SECRET)
        twitter_stream = TwitterStream(auth=oauth)
        twitter = Twitter(auth=oauth)
        x = []
        for i in range(0,1):
            #Read tweets with #jobsearch
            iterator = twitter_stream.statuses.filter(track="#jobsearch", language = "en")
            x.append(iterator)
            
        #classify tweets based on sentiments    
        df= Classify_tweet(x)
        return df
        


In [8]:

def Classify_tweet(x):
        
        collectobj = []
        for iterator in x:
            for tweet in iterator:
                if 'user' in tweet.keys() and tweet['user']['location']:
                    #Get probability of a tweet being positive, negative or neutral
                    topic = getprobtop(tweet['text'])
                    obj = {'id' : tweet['user']['id'], 'User_name' : tweet['user']['screen_name'], \
                'Text': tweet['text'],'location' : tweet['user']['location'], \
                'status' : topic}
                    print("Returning from prob")
                    collectobj.append(obj)

        table = json_normalize(collectobj)
        return(table)




In [17]:
def getprobtop(text):
        #use sentiment analyzer to get scores based on sentiments
        text = text.encode('utf8')
        text = text.decode().split('@')

        text = [x for x in text if '#jobsearch' in x.lower()]
        text = ' '.join(text)
        text  = ' '.join([x for x in text.split(' ') if '#jobsearch' not in x.lower()])
        text  = ' '.join([x for x in text.split(' ') if '#' not in x.lower()])
        t = re.match('(.*?)http.*?\s?(.*?)', text)
        if t:
            text = t.group(1)
        score = sa.polarity_scores(text.lower())
        score = {'neg' : score['neg'],'pos' : score['pos'],'neu' : score['neu']}
        key = max(score,key=score.get)
        value = score[key]
        if key == 'neu' and (score['pos']<=0.0 and score['neg']<=0.0):
            key = key;value = value
        else:
            del score['neu']
            key = max(score,key=score.get)
            value = score[key]
        print (text)
        print (key,value)
        return key



In [10]:
def main():
        #Sparksession builder
        spark = SparkSession.builder \
                 .master("local") \
                 .appName("Word Count") \
                 .config("spark.some.config.option", "some-value") \
                 .getOrCreate()
        
        for i in range(1,2):
            print(i)
            q = retrieve()
            spark.createDataFrame(q).write.parquet("infile.parquet") #Outputs the data to local FS as a parquet file
            df = spark.read.parquet("infile.parquet")
            df.write.csv("outfile.csv")
            time.sleep(15*15)
        

In [ ]:
if __name__ == '__main__':
    print("Start of extraction")
    main()